# ETL OUALYON

In [134]:
import pandas as pd
import requests as rq
import json
import re
import MapCategory as CategoryMap
import MapFiltreLyon as FiltreLyonMap

In [135]:
dataframe = pd.DataFrame(columns=['Nom', 'Commune','Adresse','Code postal','Type','Coord_X', 'Coord_Y'])

#### Import de Sport Gouv

In [136]:
response = rq.get("https://equipements.sports.gouv.fr/api/records/1.0/search/?dataset=data-es&q=&refine.code_dept=69&rows=10000&facet=adresse")
data = response.json()
for record in data['records']:
    rc = record['fields']
    if 'adresse' in rc:
        record_adresse = rc['adresse']
    else:
        record_adresse = str(rc['coordgpsy']) + '° N, ' + str(rc['coordgpsx']) + '° E'
    dataframe.loc[len(dataframe.index)] = [rc['nominstallation'],rc['nom_commune'],record_adresse,rc['codepostal'],rc['typequipement'],rc['coordgpsx'],rc['coordgpsy']]

#### Import de Data Grand Lyon

In [137]:
response = rq.get("https://download.data.grandlyon.com/ws/rdata/urbalyon.recenseqptsport/all.json?maxfeatures=10000&start=1")
data = response.json()
for record in data['values']: 
    if 'InsAdresse' in record['installation_details']:
        record_adresse = record['installation_details']['InsAdresse']
    else:
        record_adresse = str(record['lat']) + '° N, ' + str(record['lon']) + '° E'
    dataframe.loc[len(dataframe.index)] = [record['nom'],record['commune'],record_adresse,record['installation_details']['InsCodePostal'],record['type'],record['lon'],record['lat']]

In [138]:
# dataframe

#### Traitement des données

In [139]:
# dataframe = dataframe.drop_duplicates() # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
dataframe.drop_duplicates(inplace=True)
dataframe.drop_duplicates(subset=['Type','Coord_X','Coord_Y'],inplace=True) # Peut supprimer des entrées en trop dans la Category "Autres"
dataframe.drop_duplicates(subset=['Nom','Adresse','Code postal','Type'],inplace=True)
# dataframe

In [140]:
# Gestion des points au même endroit TODO
# dataframe.sort_values(by=['Coord_X', 'Coord_Y'])

Filtre sur Lyon

In [141]:
# Ajout d'une colonne "Commune = Lyon" avec des valeurs booléens  : True si la valeur de Commune est Lyon ef Nan sinon
FiltreLyonmap = FiltreLyonMap.filtreLyon
dataframe["IsLyon"] = dataframe["Commune"].map(FiltreLyonmap)
# On supprime la colonne qui a été ajouté "Commune=Lyon"
dataframe = dataframe[dataframe["IsLyon"] == True].copy()
dataframe = dataframe.drop(columns='IsLyon')
# dataframe

Filtre lieux publics

In [142]:
# On supprime les entrées contenant "college", "lycee", ...
dataframe = dataframe.loc[~dataframe['Nom'].str.contains(r'(college|collège|lycée|lycee|ecole)', case=False, na=False)]
# dataframe

C:\Users\Theo\AppData\Local\Temp\ipykernel_7468\4294668227.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataframe = dataframe.loc[~dataframe['Nom'].str.contains(r'(college|collège|lycée|lycee|ecole)', case=False, na=False)]


Catégorisation des sports

In [143]:
category_map = CategoryMap.categories

dataframe['Category'] = dataframe['Type'].map(category_map)

In [144]:
# dataframe["Commune"].unique()

In [145]:
dataframe

,Nom,Commune,Adresse,Code postal,Type,Coord_X,Coord_Y,Category
52,Parc de Parilly,Bron,36 Boulevard Emile Bollaert,69500,Stade d’athlétisme,4.894720,45.717990,Athlétisme
53,Parc de Parilly,Bron,36 Boulevard Emile Bollaert,69500,Terrain de football,4.897040,45.720450,FootBall
54,Parc de Parilly,Bron,36 Boulevard Emile Bollaert,69500,Terrain de basket-ball,4.895700,45.721680,BasketBall
57,7 Fitness,Bron,7 rue Salvador Allende,69500,Salle de cours collectifs,4.931818,45.743478,Salle de sport
58,Stade Léo Lagrange,Bron,rue Marcel Cerdan,69500,Terrain de football,4.925910,45.742140,FootBall
...,...,...,...,...,...,...,...,...
5990,Sale de remise en forme,Villeurbanne,1 rue de charny,69100,Salle de cours collectifs,4.889294,45.754860,Salle de sport
6758,Terrain basket-ball,Villeurbanne,rue lafontaine,69100,Terrain de basket-ball,4.880722,45.760173,BasketBall
7522,terrain de football,Lyon 9e Arrondissement,15 Quai de la gare d'eau,69009,Terrain de football,4.808480,45.780900,FootBall
8051,Salle de musculation,Villeurbanne,111 rue de verdun,69100,Salle d'haltérophilie,4.897147,45.778454,Salle de sport


#### Export en csv

In [146]:
dataframe.to_csv('activities.csv',sep=";")

#### Export en GeoJSON

In [147]:
# Dico python vide :
GeoJson = {
  "type": "FeatureCollection",
  "features": []
}

In [148]:
# Alimentation des features
for index, row in dataframe.iterrows():
  feature = {
    "type": "Feature",
    "properties": {
      "Name": row["Nom"],
      "Commune" : row["Commune"],
      "Adresse" : row["Adresse"],
      "Code Postal" : row["Code postal"],
      "Type" : row["Type"],
      "Category" : row["Category"]
    },
    "geometry": {
      "coordinates": [
        row["Coord_X"],
        row["Coord_Y"]
      ],
      "type": "Point"
    }
  }
  
  GeoJson["features"].append(feature)


In [149]:
# convertion en JSON:
GeoJson = json.dumps(GeoJson)

In [150]:
# Création du fichier "GeoActivities.json"
with open("Activities.geojson", "w") as outfile:
    outfile.write(GeoJson)